In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from broverse import Action, Flow, End

In [3]:
from broverse.bedrock import BedrockChat

In [354]:
import json
import yaml

In [119]:
class Start(Action):
    def __init__(self, message, next_action):
        super().__init__()
        self.message = message
        self.next_action = next_action
        
    def run(self, shared):
        print(self.message)
        shared['action'] = self.next_action
        return shared

In [398]:
class Input(Action):
    def run(self, shared):
        user_input = input("You:")
        shared['user_input'] = user_input
        shared['action'] = 'continue'
        return shared
    
class SystemCommand(Action):
    def __init__(self, ):
        super().__init__()
        self.commands = [
            f"/{command}"
            for command
            in ["clear_user_input", "exit", "list"]
        ]
    def get_slash_command(self, user_input:str):
        if user_input.startswith('/clear_user_input'):
            return 'clear_user_input'
        if user_input.startswith('/exit'):
            return 'end'
        if user_input.startswith('/list'):
            print("Available commands are:")
            for command in self.commands:
                print(f"\t- {command}")
            return 'continue'
        return 'continue'
    
    def run(self, shared):
        user_input = shared.get('user_input', 'no input')
        action = self.get_slash_command(user_input)
        if action == 'end':
            shared['action'] = 'end'
        elif action != 'continue':
            shared['tool'] = action
            shared['action'] = 'system_command'
        else:
            shared['action'] = 'continue'
        return shared

class ToolSelector(Action):
    def __init__(self, system_prompt:str, model:BedrockChat, tools:dict):
        super().__init__()
        self.system_prompt = system_prompt
        self.model = model
        self.tools = tools

    def format_tools_for_selector(self):
        """Extract only tool name and description for ToolSelector"""
        formatted = []
        for name, tool_info in self.tools.items():
            description = tool_info['definition']['description']
            formatted.append(f"{name}: {description}")
        return formatted
    
    def convert_tool_str_to_dict(self, tool, codeblock='json'):
        tool = tool.split("```"+codeblock)
        if len(tool)>1:
            tool = tool[1]
        else:
            tool = tool[-1]
        tool = tool.split("```")[0].strip()
        print(tool)
        if codeblock=='json':
            return json.loads(tool)
        return yaml.safe_load(tool)
    
    def run(self, shared):
        user_input = shared.get("user_input", "")
        tool_list = self.format_tools_for_selector()
        tool_list = "\t- ".join(tool_list)
        prompt = f"USER_INPUT: \n\n{user_input}\n\n"
        tool = self.model.run(
            system_prompt=self.system_prompt.format(tools=tool_list),
            messages=[self.model.UserMessage(text=prompt)]
        )
        # print(tool)
        tool = self.convert_tool_str_to_dict(tool, codeblock='yaml').get('tool', None)
        shared['tool'] = tool
        shared['action'] = 'tool calling' if tool in self.tools else None
        
        return shared

In [424]:
import inspect
import typing
from typing import get_type_hints
import json

class Tools(Action):
    def __init__(self, system_prompt:str, model:BedrockChat, tools:list):
        super().__init__()
        self.system_prompt = system_prompt
        self.model = model
        self.tools = self.register_tools(tools)

    def convert_to_tool(self, func)->dict:
        sig = inspect.signature(func)
        type_hints = get_type_hints(func)

        parameters = {
            "type": "object",
            "properties": {},
            "required": []
        }

        for name, param in sig.parameters.items():
            param_type = type_hints.get(name, str)
            param_schema = {"type": self.python_type_to_json_type(param_type)}
            parameters["properties"][name] = param_schema
            if param.default is param.empty:
                parameters["required"].append(name)

        tool = {
            "name": func.__name__,
            "description": func.__doc__ or "",
            "parameters": parameters
        }

        return tool


    def python_type_to_json_type(self, py_type):
        origin = typing.get_origin(py_type) or py_type

        if origin in [int]:
            return "integer"
        elif origin in [float]:
            return "number"
        elif origin in [bool]:
            return "boolean"
        elif origin in [str]:
            return "string"
        elif origin in [list, tuple]:
            return "array"
        elif origin in [dict]:
            return "object"
        else:
            return "string"  # fallback
        
    def get_params(self, tool):
        return self.convert_to_tool(tool)
        
    def register_tools(self, tools):
        _tools = {}
        for tool, action in tools:
            tool_definition = self.get_params(tool)
            name = tool_definition.get("name", None)
            _tools[name] = {
                "definition": tool_definition,
                "tool": tool,
                "action": action
            }
        return _tools

    def parse_str_to_json(self, params):
        params = params.split("```json")
        if len(params)>1:
            params = params[1]
        else:
            params = params[0]
        params = params.split("```")[0]
        return json.loads(params)
        
    def run(self, shared):
        user_input = shared.get('user_input', 'no input')
        tool_name = shared.get('tool', 'no tool')
        tool = self.tools.get(tool_name, {})
        definition = tool.get('definition', None)
        action = tool.get('action', None)
        tool = tool.get('tool', None)
        prompt = f"USER_INPUT: \n\n{user_input}\n\n"
        params = self.model.run(
            system_prompt=self.system_prompt.format(definition=json.dumps(definition)), 
            messages=[self.model.UserMessage(text=prompt)]
            )
        params = self.parse_str_to_json(params)
        if "shared" in params:
            params.pop("shared")
        response = tool(shared=shared, **params)
        shared['tool_res'] = response
        shared['action'] = action
        return shared

def add(a:int, b:int, **kwargs):
    """This function can add two numbers together"""
    result = a+b
    print(f"a+b is: {a}+{b}={result}")
    return result

def clear_user_input(shared:dict, **kwargs):
    """Clear user input"""
    shared['user_input'] = ''
    print("user_input is cleared")
    return None

In [425]:
# tool_selector_prompt = """\
# You are a tool selection expert.

# Your job is to analyze the USER_INPUT and select the single best matching tool from TOOLS.
# If you can't find the best matching tool, return null.
# TOOLS:
# {tools}

# Return only the tool name, nothing else, in JSON codeblock with a specified JSON format:
# ```json
# {{"tool": "tool name"}}
# ```

# Examples:
# USER_INPUT: "add 5 and 3"
# OUTPUT: 
# ```json
# {{"tool": "add"}}
# ```

# USER_INPUT: "clear my input"
# OUTPUT:
# ```json
# {{"tool": "clear_user_input"}}
# ```

# USER_INPUT: "do something and the do something is not in TOOLS"
# OUTPUT:
# ```json
# {{"tool": null}}
# ```
# """


In [426]:
tool_selector_prompt = """\
You are a tool selection expert.

Your job is to analyze the USER_INPUT and select the single best matching tool from TOOLS.
If you can't find the best matching tool, return null.
TOOLS:
{tools}

Return only the tool name, nothing else, in YAML codeblock with a specified YAML format:
```yaml
tool: tool name
```

Examples:
USER_INPUT: "add 5 and 3"
OUTPUT: 
```yaml
tool: add
```

USER_INPUT: "clear my input"
OUTPUT:
```yaml
tool: clear_user_input
```

USER_INPUT: "do something and the do something is not in TOOLS"
OUTPUT:
```yaml
tool: null
```
"""


In [427]:
import yaml
yaml_str = """
tool: null
""".strip()
yaml.safe_load(yaml_str)

{'tool': None}

In [428]:
tool_selector_prompt.format(tools="Test")

'You are a tool selection expert.\n\nYour job is to analyze the USER_INPUT and select the single best matching tool from TOOLS.\nIf you can\'t find the best matching tool, return null.\nTOOLS:\nTest\n\nReturn only the tool name, nothing else, in YAML codeblock with a specified YAML format:\n```yaml\ntool: tool name\n```\n\nExamples:\nUSER_INPUT: "add 5 and 3"\nOUTPUT: \n```yaml\ntool: add\n```\n\nUSER_INPUT: "clear my input"\nOUTPUT:\n```yaml\ntool: clear_user_input\n```\n\nUSER_INPUT: "do something and the do something is not in TOOLS"\nOUTPUT:\n```yaml\ntool: null\n```\n'

In [429]:
tool_prompt = """\
You are a function argument extractor. 
Extract USER_INPUT required to call the function. Return a JSON object that matches the parameters schema exactly. Only include the keys that are required.

Given the following function definition:
DEFINITION
```json
{definition}
```

Respond only with the JSON object.
And the following user input:
""".strip()

In [430]:
json.dumps({"tool":None})

'{"tool": null}'

In [431]:
tools = Tools(
    system_prompt=tool_prompt,
    model=BedrockChat(),
    tools=[
        (clear_user_input, 'continue'),
        (add, 'continue')
    ]
)

tool_selector_action = ToolSelector(
    system_prompt=tool_selector_prompt,
    model=BedrockChat(),
    tools=tools.tools
)

In [432]:
start_action = Start(message="Start Flow", next_action="input")
input_action = Input()
system_command_action = SystemCommand()
end_action = End(message="End Flow")

In [433]:
start_action - "input" >> input_action
input_action - "continue" >> system_command_action
system_command_action - "continue" >> tool_selector_action
tool_selector_action - "tool calling" >> tools
system_command_action - "system_command" >> tools
tools - "continue" >> input_action
system_command_action - "end" >> end_action

In [434]:
flow = Flow(start_action=start_action)

In [436]:
shared = {}
flow.run(shared=shared)
shared

Start Flow
tool: add
a+b is: 1+1=2
tool: add
a+b is: 1+1=2
tool: add
a+b is: 1+2=3
tool: add
a+b is: 1+1=2
tool: add
a+b is: 1+1=2
tool: null


{'action': None, 'user_input': '', 'tool': None, 'tool_res': 2}

In [437]:
shared

{'action': None, 'user_input': '', 'tool': None, 'tool_res': 2}

In [1]:
import yaml
with open("./examples/tool_calling/prompts/tool_selector.yaml", "r") as f:
    yaml_str = yaml.safe_load(f)

In [2]:
yaml_str

{'persona': 'You are a tool selection expert.',
 'instructions': ['Analyze the USER_INPUT and select the single best matching tool from TOOLS',
  "If you can't find the best matching tool, return null"],
 'tools_placeholder': '{tools}',
 'structured_output': {'format': 'YAML codeblock',
  'schema': '```yaml\ntool: tool name\n```\n'},
 'examples': [{'user_input': 'add 5 and 3',
   'output': '```yaml\ntool: add\n```\n'},
  {'user_input': 'clear my input',
   'output': '```yaml\ntool: clear_user_input\n```\n'},
  {'user_input': 'do something and the do something is not in TOOLS',
   'output': '```yaml\ntool: null\n```'}]}

In [4]:
def convert_yaml_to_prompt(filepath):
    with open(filepath, "r") as f:
        data = yaml.safe_load(f)

    prompt = []
    for k, v in data.items():
        _prompt = f"{k.upper()}:\n{v}"
        prompt.append(_prompt)
    return prompt

In [1]:
filepath = "./examples/tool_calling/prompts/tool_selector.yaml"
# prompt = convert_yaml_to_prompt(filepath)
# print("\n".join(prompt))

In [2]:
with open(filepath, 'r') as f:
    data = f.read()
print(data)

persona: "You are a tool selection expert."

instructions:
  - "Analyze the USER_INPUT and select the single best matching tool from TOOLS"
  - "If you can't find the best matching tool, return null"

tools_placeholder: {tools}

structured_output:
  format: "YAML codeblock"
  schema: |
    ```yaml
    tool: tool name
    ```

examples:
  - user_input: "add 5 and 3"
    output: |
      ```yaml
      tool: add
      ```
  - user_input: "clear my input"
    output: |
      ```yaml
      tool: clear_user_input
      ```
  - user_input: "do something and the do something is not in TOOLS"
    output: |
      ```yaml
      tool: null
      ```


In [5]:
print(data.format(tools="""\n\t- a: b\n\t- c:d"""))

persona: "You are a tool selection expert."

instructions:
  - "Analyze the USER_INPUT and select the single best matching tool from TOOLS"
  - "If you can't find the best matching tool, return null"

tools_placeholder: 
	- a: b
	- c:d

structured_output:
  format: "YAML codeblock"
  schema: |
    ```yaml
    tool: tool name
    ```

examples:
  - user_input: "add 5 and 3"
    output: |
      ```yaml
      tool: add
      ```
  - user_input: "clear my input"
    output: |
      ```yaml
      tool: clear_user_input
      ```
  - user_input: "do something and the do something is not in TOOLS"
    output: |
      ```yaml
      tool: null
      ```


In [5]:
from typing import Any, Dict
from broverse import Action, Flow, Start, End

class Test1(Action):
    def run(self, shared):
        return shared
    
class Test2(Action):
    def run(self, shared):
        return shared
    
t1 = Test1()
t2 = Test2()
start1 = Start(message='Start 1')
end1 = End(message='End 1')

# start1 >> t1
# t1 >> t2
# t2 >> end1
start1 >> t1 >> t2 >> end1
flow1 = Flow(start_action=start1, name='Start flow1')

In [6]:
print(flow1.to_mermaid())

```mermaid
flowchart TD
    Start_flow1 -->|default| Test1
    Test1 -->|default| Test2
    Test2 -->|default| End
```


In [7]:
class Test3(Action):
    def run(self, shared):
        return shared
    
class Test4(Action):
    def run(self, shared):
        return shared
    
t3 = Test3()
t4 = Test4()
start2 = Start(message='Start 2')
end2 = End(message='End 2')

start2 >> t3 >> t4 >> end2

flow2 = Flow(start_action=start2, name='Start flow2')

In [8]:
from typing import Any, Dict
import random

class Router(Action):
    def validate_next_action(self, shared: Dict[str, Any]) -> str:
        return random.choice(['flow1', 'flow2'])
    
    def run(self, shared):
        return shared
class Test5(Action):
    def run(self, shared):
        return shared

t5 = Test5()
router = Router()
start_flow = Start(message="Start Flow")
end_flow = End(message="End Flow")

start_flow >> router
router - "flow1" >> flow1
router - "flow2" >> flow2
flow1 >> flow2 >> t5 >> end_flow

In [9]:
master_flow = Flow(start_action=start_flow, name='master_flow')

In [10]:
master_flow.save_mermaid('flowtoflow.md')
chart = master_flow.to_mermaid()
print(chart)

```mermaid
flowchart TD
    master_flow -->|default| Router
    Router -->|flow1| Start_flow1
    Start_flow1 -->|default| Start_flow2
    Start_flow2 -->|default| Test5
    Test5 -->|default| End
    Router -->|flow2| Start_flow2
```


In [11]:
master_flow.successors

{}

In [12]:
master_flow.start_action.successors

{'default': <__main__.Router at 0x263c62be390>}

In [13]:
master_flow.run({})

Start Flow
Running action: Router
Running action: Flow
Start 1
Running action: Test1
Running action: Test2
End 1
Running action: Flow
Start 2
Running action: Test3
Running action: Test4
End 2
Running action: Test5
End Flow


'default'

In [1]:
from broverse.action_async import AsyncAction, AsyncStart, AsyncEnd
from broverse.flow_async import AsyncFlow, AsyncParallelFlow
import asyncio

class LLMCall(AsyncAction):
    async def _run_async(self, shared):
        print("Making LLM API call...")
        await asyncio.sleep(2)  # Simulate 2 second LLM call
        shared['llm_response'] = "Hello from AI!"
        return shared


# Build flow
start = AsyncStart("Starting AI Agent")
llm = LLMCall(max_retries=3, wait=1)
end = AsyncEnd("Agent Complete")

start >> llm 
llm >> end
flow = AsyncFlow(start)

shared_state = {}
# Run async
result = await flow.run_async(shared_state)


Starting AI Agent
Running async action: LLMCall
Making LLM API call...
Agent Complete


In [1]:
from broverse import state, Action, load_config

In [2]:
class Test(Action):
    def run(self, shared):
        self.print("abc")
        return shared

In [4]:
load_config('config.yaml')

{'debug': False}

In [5]:
Test().run({})

{}

In [4]:
state.set('debug', False)

In [8]:
state.get('debug')

True

In [7]:
state.set('debug', True)

In [1]:
from typing import Annotated

In [5]:
def test(a:Annotated[int, "a number"]):
    return a

In [6]:
test

<function __main__.test(a: typing.Annotated[int, 'a number'])>

In [26]:
prompt = """\
Select the best matching tool based on user's intent. 

Given the below tools, choose one best match:
TOOLS:

{tools}
""".strip()

In [27]:
def register_tools(tools):
    return {tool.__name__:tool.__doc__ for tool in tools}

In [28]:
tool_obj = register_tools([create_appointment])
tool_obj

{'create_appointment': '\n    <|start_summary|>\n        Add or create an appointment\n    <|end_summary|>\n\n    Args:\n        event (str) : the name of event\n        year (int) : the year of the event\n        month (int) : the month of the event\n        day (int) : the day of the event\n\n    Return:\n        str\n    '}

In [29]:
print("\n".join([f"\t- {name} : {definition}" for name, definition in tool_obj.items()]))

	- create_appointment : 
    <|start_summary|>
        Add or create an appointment
    <|end_summary|>

    Args:
        event (str) : the name of event
        year (int) : the year of the event
        month (int) : the month of the event
        day (int) : the day of the event

    Return:
        str
    


In [30]:
print(prompt.format(tools="\n".join([f"\t- {name} : {definition}" for name, definition in tool_obj.items()])))

Select the best matching tool based on user's intent. 

Given the below tools, choose one best match:
TOOLS:

	- create_appointment : 
    <|start_summary|>
        Add or create an appointment
    <|end_summary|>

    Args:
        event (str) : the name of event
        year (int) : the year of the event
        month (int) : the month of the event
        day (int) : the day of the event

    Return:
        str
    


In [31]:
import re

def extract_summary(text: str) -> str:
    match = re.search(r'<\|start\|>(.*?)<\|end\|>', text, re.DOTALL)
    return match.group(1).strip() if match else ""

In [40]:
def add(a:float, b:float)->float:
    """
    <|start|>
        Add two numbers together. 
        User may call `add` or `plus` or `+` to initialize this function
    <|end|>
    Args:
        a (float) : an input number
        b (float) : another input number
    Return:
        float : the result of a+b
    """
    return a+b

In [ ]:
def extract_tools(funcs)->list[str]:
    """Use this function for ToolSelector"""
    prompt = []
    for func in funcs:
        name = func.__name__
        definition = func.__doc__.strip()
        match = re.search(r'<\|start\|>(.*?)<\|end\|>', definition, re.DOTALL)
        text = match.group(1).strip() if match else ""
        text = re.sub(r'\s+', ' ', text).strip()
        prompt.append(f"{name}: {text}")
    return prompt

In [42]:
extract_tool(add)

'add: Add two numbers together. User may call `add` or `plus` or `+` to initialize this function'

In [47]:
import inspect
import re

def generate_extraction_prompt(func):
    """Generate a prompt to extract function parameters from user input."""
    sig = inspect.signature(func)
    doc = inspect.getdoc(func) or ""
    
    # Extract description between <|start|> and <|end|>
    desc_match = re.search(r'<\|start\|>(.*?)<\|end\|>', doc, re.DOTALL)
    description = desc_match.group(1).strip() if desc_match else ""
    description = re.sub(r'\s+', ' ', description).strip()
    
    # Build parameter info
    params = []
    for name, param in sig.parameters.items():
        param_type = param.annotation.__name__ if param.annotation != inspect.Parameter.empty else "any"
        params.append(f"- {name} ({param_type})")
    
    # Build YAML template
    yaml_template = "\n".join(f"{name}: <value>" for name in sig.parameters.keys())
    
    prompt = f"""Extract parameters for the {func.__name__} function from user input and return them in YAML format.

Function: {func.__name__}{sig}
Description: {description}

Parameters:
{chr(10).join(params)}

Return only the YAML format:
```yaml
{yaml_template}
User input: {{user_input}}"""
    return prompt

In [48]:
print(generate_extraction_prompt(add))

Extract parameters for the add function from user input and return them in YAML format.

Function: add(a: float, b: float) -> float
Description: Add two numbers together. User may call `add` or `plus` or `+` to initialize this function

Parameters:
- a (float)
- b (float)

Return only the YAML format:
```yaml
a: <value>
b: <value>
User input: {user_input}


In [50]:
def add_multiple_numbers(numbers:list)->float:
    """
    <|start|>Add the list of numbers. User may use keywords like `add`, `plus`, `+`<|end|>
    """
    return sum(numbers)
print(generate_extraction_prompt(add_multiple_numbers))

Extract parameters for the add_multiple_numbers function from user input and return them in YAML format.

Function: add_multiple_numbers(numbers: list) -> float
Description: Add the list of numbers. User may use keywords like `add`, `plus`, `+`

Parameters:
- numbers (list)

Return only the YAML format:
```yaml
numbers: <value>
User input: {user_input}


In [2]:
from broverse.tools import generate_extract_parameters_prompt, list_tools

In [ ]:
def add_multiple_numbers(numbers:list[float])->float:
    """
    <|start|>Add the list of numbers. User may use keywords like `add`, `plus`, `+`<|end|>
    Args:
        numbers (list) : a list of numbers
    Return:
        float : the result of sum(numbers)
    """
    return sum(numbers)

def add(a:float, b:float)->float:
    """
    <|start|>
        Add two numbers together. 
        User may call `add` or `plus` or `+` to initialize this function
    <|end|>
    Args:
        a (float) : an input number
        b (float) : another input number
    Return:
        float : the result of a+b
    """
    return a+b

def create_appointment(event:str, year:int, month:int, day:int):
    """
    <|start|>
        Add or create an appointment
    <|end|>
    
    Args:
        event (str) : the name of event
        year (int) : the year of the event
        month (int) : the month of the event
        day (int) : the day of the event
    
    Return:
        str
    """
    return "done"

In [21]:
prompt = """\
Select the best matching tool based on user's intent. 

Given the below tools, choose one best match:
TOOLS:

{tools}
""".strip()

In [22]:
tool_list = list_tools(funcs=[add, add_multiple_numbers, create_appointment])

In [ ]:
print(prompt.format(tools="\n".join(tool_list)))

Select the best matching tool based on user's intent. 

Given the below tools, choose one best match:
TOOLS:

	- add: Add two numbers together. User may call `add` or `plus` or `+` to initialize this function
	- add_multiple_numbers: Add the list of numbers. User may use keywords like `add`, `plus`, `+`
	- create_appointment: Add or create an appointment


In [35]:
print(generate_extract_parameters_prompt(add))

Extract parameters for the add function from user input and return them in YAML format.

Function: add(a: float, b: float) -> float
Description: Add two numbers together. User may call `add` or `plus` or `+` to initialize this function

Parameters:
- a (float)
- b (float)

Return only the YAML format:
```yaml
a: <value>
b: <value>
```
User input: {user_input}


In [33]:
print(generate_extract_parameters_prompt(add_multiple_numbers))

Extract parameters for the add_multiple_numbers function from user input and return them in YAML format.

Function: add_multiple_numbers(numbers: list) -> float
Description: Add the list of numbers. User may use keywords like `add`, `plus`, `+`

Parameters:
- numbers (list)

Return only the YAML format:
```yaml
numbers: [<value>]
```
User input: {user_input}


In [32]:
import inspect
import typing
from typing import get_type_hints, Callable
import re
def generate_extract_parameters_prompt(func):
    """Generate a prompt to extract function parameters from user input."""
    sig = inspect.signature(func)
    doc = inspect.getdoc(func) or ""
    
    # Extract description between <|start|> and <|end|>
    desc_match = re.search(r'<\|start\|>(.*?)<\|end\|>', doc, re.DOTALL)
    description = desc_match.group(1).strip() if desc_match else ""
    description = re.sub(r'\s+', ' ', description).strip()
    
    # Build parameter info
    params = []
    for name, param in sig.parameters.items():
        param_type = param.annotation.__name__ if param.annotation != inspect.Parameter.empty else "any"
        params.append(f"- {name} ({param_type})")
    
    # Build YAML template with proper formatting for lists
    yaml_lines = []
    for name, param in sig.parameters.items():
        if param.annotation != inspect.Parameter.empty:
            origin = typing.get_origin(param.annotation) or param.annotation
            if origin in [list, tuple]:
                yaml_lines.append(f"{name}: [<value>]")
            else:
                yaml_lines.append(f"{name}: <value>")
        else:
            yaml_lines.append(f"{name}: <value>")
    
    yaml_template = "\n".join(yaml_lines)
    
    prompt = f"""Extract parameters for the {func.__name__} function from user input and return them in YAML format.

Function: {func.__name__}{sig}
Description: {description}

Parameters:
{chr(10).join(params)}

Return only the YAML format:
```yaml
{yaml_template}
```
User input: {{user_input}}"""
    return prompt

In [36]:
def validate_parameters(params_dict, func):
    """Validate extracted parameters against function signature"""
    sig = inspect.signature(func)
    
    # Check required parameters
    missing = []
    for name, param in sig.parameters.items():
        if param.default is param.empty and name not in params_dict:
            missing.append(name)
    
    if missing:
        return False, f"Missing required parameters: {', '.join(missing)}"
    
    # Check parameter types (basic validation)
    type_hints = get_type_hints(func)
    for name, value in params_dict.items():
        if name in type_hints:
            expected_type = type_hints[name]
            origin = typing.get_origin(expected_type) or expected_type
            
            if origin == list and not isinstance(value, list):
                return False, f"Parameter '{name}' should be a list"
            elif origin == int and not isinstance(value, int):
                return False, f"Parameter '{name}' should be an integer"
            elif origin == float and not isinstance(value, (int, float)):
                return False, f"Parameter '{name}' should be a number"
    
    return True, None


In [55]:
def validate_parameters(params_dict, func):
    """Validate extracted parameters against function signature"""
    sig = inspect.signature(func)
    
    # Check required parameters
    missing = []
    for name, param in sig.parameters.items():
        if param.default is param.empty and name not in params_dict:
            missing.append(name)
    
    if missing:
        return False, f"Missing required parameters: {', '.join(missing)}"
    
    # Check parameter types
    type_hints = get_type_hints(func)
    for name, value in params_dict.items():
        if name in type_hints:
            expected_type = type_hints[name]
            
            # Handle list[float] specifically
            if hasattr(expected_type, '__origin__') and expected_type.__origin__ is list:
                if not isinstance(value, list):
                    return False, f"Parameter '{name}' should be a list"
                
                # Check list elements
                if hasattr(expected_type, '__args__') and expected_type.__args__:
                    element_type = expected_type.__args__[0]
                    if element_type is float:
                        for i, item in enumerate(value):
                            if not isinstance(item, (int, float)):
                                return False, f"Parameter '{name}[{i}]' should be a number, got {type(item).__name__}"
    
    return True, None



In [56]:
from typing import List

def add_multiple_numbers(numbers:List[float])->float:
    """
    <|start|>Add the list of numbers. User may use keywords like `add`, `plus`, `+`<|end|>
    Args:
        numbers (list) : a list of numbers
    Return:
        float : the result of sum(numbers)
    """
    return sum(numbers)

In [57]:
params_dict = {"numbers": [1,2,"3"]}
validate_parameters(params_dict, add_multiple_numbers)

(False, "Parameter 'numbers[2]' should be a number, got str")

In [62]:
user_input = "/exit"
user_input.startswith("/")

True